In [1]:
import requests
import lxml.html
from tqdm.notebook import tqdm
import json
from datetime import datetime

In [2]:
base_url = "https://www.heise.de/newsticker/wirtschaft/"
page_url = lambda seite: f"https://www.heise.de/newsticker/wirtschaft/seite-{seite}/"

# Get all article urls

In [3]:
response = requests.get(base_url)
html = lxml.html.document_fromstring(response.text)
last_page = int(html.xpath('//ol[@class="a-pagination__items a-pagination__items--word"]/li/a/@href')[-2].split("-")[-1][:-1])

In [11]:
all_article_urls = []
for i in tqdm(range(last_page)):
    response = requests.get(page_url(i+1))
    html = lxml.html.document_fromstring(response.text)
    article_urls = html.xpath('//div[@class="article-index"]/article/a/@href')
    all_article_urls += article_urls

  0%|          | 0/922 [00:00<?, ?it/s]

## Remove techbook articles

In [41]:
for url in all_article_urls:
    if url.startswith("https:") :
        all_article_urls.remove(url)
print(len(all_article_urls))

13662


## Save

In [31]:
with open("urls.txt", "w") as output:
    for url in all_article_urls:
        output.write("%s\n" % url)

In [3]:
with open("urls.txt", "r") as input_file:
    all_article_urls = input_file.readlines()
    
all_article_urls = [url.strip() for url in all_article_urls]

In [55]:
all_article_urls

['/news/Bundeskartellamt-prueft-Anmeldepflicht-von-Facebook-Deal-6146222.html',
 '/news/Klimaneutralitaet-bei-der-Schifffahrt-Der-Weg-ist-noch-weit-6146078.html',
 '/news/Vodafone-Deutschland-Glasfaseranschluesse-und-Roaming-treiben-Umsatz-6146051.html',
 '/news/Verkaufszahlen-von-Hybrid-und-Elektroautos-steigen-in-der-EU-stark-an-6145981.html',
 '/news/Snap-legt-starkes-Nutzerwachstum-hin-und-verringert-Verluste-6145912.html',
 '/hintergrund/Mercedes-setzt-auf-Electric-only-6145955.html',
 '/news/Twitter-uebertrifft-Erwartungen-Aktie-steigt-6145896.html',
 '/news/Studie-E-Autos-verbessern-Klimabilanz-weiter-gegenueber-Verbrennern-6145800.html',
 '/news/Notebook-Boom-in-der-Corona-Pandemie-treibt-Intel-Geschaeft-weiter-an-6145884.html',
 '/news/BioNTech-Pfizer-Abfuell-Deal-fuer-mRNA-Impfstoff-mit-Suedafrika-stoesst-auf-Kritik-6145676.html',
 '/news/Umwelthilfe-beklagt-katastrophale-Sammelbilanz-bei-Elektroschrott-6145525.html',
 '/news/Neuer-Vorstandschef-Intershop-waechst-profitabel-6

# Get articles

In [ ]:
articles = []
for article_url in tqdm(all_article_urls):
    
    try:
        article_url = f"https://www.heise.de{article_url}"
        response = requests.get(article_url)
        article = lxml.html.document_fromstring(response.text)
        #article = html.xpath('//div[normalize-space(@class)="a-layout__item"]/article')[0]

        article_dict = {}

        article_dict["article_url"] = article_url

        try:
            number_comments = int(article.xpath('//span[normalize-space(@class)="comment-button__text"]/text()')[0].split("(")[-1].split(")")[0])
        except ValueError:
            number_comments = 0
        article_dict["number_comments"] = number_comments

        date = datetime.strptime(article.xpath('//time/@datetime')[0],"%Y-%m-%dT%H:%M:%S%z")
        if not date:
            print(f"Fehler date {article_url}")
        article_dict["date"] = date.strftime("%m-%d-%Y, %H:%M")

        brandig = [tag.strip() for tag in article.xpath('//div[normalize-space(@class)="a-publish-info__branding"]/a/div/text()')]
        article_dict["brandig"] = brandig

        related_topics = [topic.strip() for topic in article.xpath('//li[normalize-space(@class)="topics__item"]/a/text()')]
        article_dict["related_topics"] = related_topics

        authors = article.xpath('//li[normalize-space(@class)="a-creator__name"]/text()')
        article_dict["authors"] = authors

        heading = article.xpath('//h1[normalize-space(@class)="a-article-header__title"]/text()')[0].strip()
        if not heading:
            print(f"Fehler Heading {article_url}")
        article_dict["heading"] = heading

        lead = article.xpath('//p[normalize-space(@class)="a-article-header__lead"]/text()')[0].strip()
        if not lead:
            print(f"Fehler Lead {article_url}")
        article_dict["lead"] = lead

        article_text_query = '//div[normalize-space(@class)="article-layout__content article-content"]/p/text() | //div[normalize-space(@class)="article-layout__content article-content"]/h3/text()'

        atricle_text = " ".join([text.strip() for text in article.xpath(article_text_query) if text.strip()])
        if not atricle_text:
            print(f"Fehler text {article_url}")
        article_dict["atricle_text"] = atricle_text

        articles.append(article_dict)
    
    except:
        print(article_url)

  0%|          | 0/13662 [00:00<?, ?it/s]

# TODO

- artikel tags verwandte
- datum
- text 